Run the following code before running this script

ROAST to get Bvals

0.5_extract_sphere_data.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score

#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense


from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor

import pickle
from keras.models import load_model

# Load prepared data

In [ ]:
dir_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/"
# dir_path = "/Users/mriworkshop/Documents/TDCS/code/tdcs_thesis/"
save_path = dir_path+"data/raw/"
img_path =  dir_path+"data/processed/"
model_path = dir_path+"models/"

## fmap mean all experiments

In [ ]:
file_mean = save_path+"fmap_mean_40.txt"
columns_mean =['exp', 'mini_exp', 'i', 'j', 'k', 'mean0', 'mean1', 'mean2', 'mean3', 'mean4']
data = np.loadtxt(file_mean);

In [ ]:
df = pd.DataFrame(data, columns=columns_mean)
df = df.astype({"exp": int, "i": int, "j": int, "k": int, "mini_exp": int})
df

# Separate -ON case

In [ ]:
file_mean = save_path+"fmap_mean_neg_amp_42.txt"
columns_mean =['exp', 'mini_exp', 'i', 'j', 'k', 'amp', 'neg', 'mean0', 'mean1', 'mean2','target']
data = np.loadtxt(file_mean);

In [ ]:
df = pd.DataFrame(data, columns=columns_mean)
df = df.astype({"exp": int, "i": int, "j": int, "k": int, "mini_exp": int, "amp":int, "neg": int})
df

In [ ]:
df.info()

# Split data

## Get one experiment

In [ ]:
df_test1 = df[(df['exp']==42) & (df['mini_exp']==3)]
# df_test2 = df[(df['exp']==41) & (df['mini_exp']==1)]
# df_test3 = df[(df['exp']==42) & (df['mini_exp']==1)]

In [ ]:
print(df_test1.shape)
# print(df_test2.shape)
# print(df_test3.shape)

## None zero

In [ ]:
df_nonzero = df[(df['mean0']!=0.0) & (df['mean1']!=0.0) & (df['mean2']!=0.0) & (df['mean3']!=0.0) & (df['mean4']!=0.0)]

In [ ]:
# For separate -ON
df_nonzero = df[(df['mean0']!=0.0) & (df['mean1']!=0.0) & (df['mean2']!=0.0)]

In [ ]:
df_nonzero

In [ ]:
df4_pred1 =  df_nonzero[(df_nonzero['exp']==42) & (df_nonzero['mini_exp']==3)]
# df4_pred2 =  df_nonzero[(df_nonzero['exp']==41) & (df_nonzero['mini_exp']==1)]
# df4_pred3 =  df_nonzero[(df_nonzero['exp']==42) & (df_nonzero['mini_exp']==1)]

In [ ]:
X4_pred1 = df4_pred1.iloc[:, 6:-1].values
# y4_pred1 = df4_pred1['theory'].values

# X4_pred2 = df4_pred2.iloc[:, 5:].values
# # y4_pred2 = df4_pred2['theory'].values

# X4_pred3 = df4_pred3.iloc[:, 5:].values
# # y4_pred3 = df4_pred3['theory'].values

In [ ]:
print(X4_pred1.shape)
# print(X4_pred2.shape)
# print(X4_pred3.shape)

## Load model

In [ ]:
file = model_path+'model_nonzero_32to38_3layer5unit_300.sav'
model = pickle.load(open(file, 'rb'))

In [ ]:
# For separate -ON
file = model_path+'best_model_nn_neg_nonzero.h5'
model = load_model(file)

# Predict

In [ ]:
df_test = df_test1
df_pred = df4_pred1

In [ ]:
X_test = X4_pred1
# y_test = y4_pred3

In [ ]:
# GO TO Output section

In [ ]:
result = model.evaluate(X_test, y_test)
print(result)

In [ ]:
test_pred = model.predict(X_test)
print(mse(test_pred, y_test))
print(mape(test_pred, y_test))

In [ ]:
np.corrcoef(test_pred.flatten(), y_test)

# Plot

### Theory vs Predict

In [ ]:
x = y_test.flatten()
y = test_pred.flatten()
m, b, r, p, st_er = stats.linregress(x,y) 

yfit = [b + m * xi for xi in x]
yisx = [0 + 1 * xi for xi in x]
plt.plot(x, yfit)
plt.plot(x, yisx)

plt.scatter(y_test, test_pred,  color='black')
plt.axis([0,100, 0, 100])
plt.xlabel("Theory (nT)")
plt.ylabel("Prediction (nT)")
plt.title("Neural Network Prediction vs Theory", fontsize=15)
# print(r, st_er)
print("r: {:.5f}, st_er: {:.6f}".format(r, st_er))
print("y = "+str(round(m,4))+"*x + "+str(round(b,4)))

# Output data

In [ ]:
test_pred = model.predict(X_test)

In [ ]:
df_pred['predict'] = test_pred

In [ ]:
df_test['predict'] = 0.00

In [ ]:
df_out = df_test[['i', 'j', 'k', 'predict']]
df_pre = df_pred[['i', 'j', 'k', 'predict']]

In [ ]:
# For separate -ON
df_out = df_test[['i', 'j', 'k', 'neg', 'predict']]
df_pre = df_pred[['i', 'j', 'k', 'neg', 'predict']]

In [ ]:
df_test

In [ ]:
for x in range(len(df_pre)):
    i = df_pre.iloc[x, :]['i'].astype(int)
    j = df_pre.iloc[x, :]['j'].astype(int)
    k = df_pre.iloc[x, :]['k'].astype(int)
    pred = df_pre.iloc[x, :]['predict']
    idx = df_out[(df_out['i']==i) & (df_out['j']==j) & (df_out['k']==k)].index
    df_out.loc[idx, 'predict']= pred

In [33]:
# For separate -ON
for x in range(len(df_pre)):
    i = df_pre.iloc[x, :]['i'].astype(int)
    j = df_pre.iloc[x, :]['j'].astype(int)
    k = df_pre.iloc[x, :]['k'].astype(int)
    neg = df_pre.iloc[x, :]['neg'].astype(int)
    pred = df_pre.iloc[x, :]['predict']
    idx = df_out[(df_out['i']==i) & (df_out['j']==j) & (df_out['k']==k) & (df_out['neg']==neg)].index
    df_out.loc[idx, 'predict']= pred

In [34]:
df_out

,i,j,k,neg,predict
720896,0,0,0,0,0.0
720897,0,0,1,0,0.0
720898,0,0,2,0,0.0
720899,0,0,3,0,0.0
720900,0,0,4,0,0.0
...,...,...,...,...,...
1081339,43,63,59,1,0.0
1081340,43,63,60,1,0.0
1081341,43,63,61,1,0.0
1081342,43,63,62,1,0.0


In [ ]:
np.savetxt(img_path+"nn_nonzero_42_predict.txt", df_out[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")

In [35]:
# For separate -ON
df_pos = df_out[df_out['neg']==0]
df_neg = df_out[df_out['neg']==1]
np.savetxt(img_path+"nn_nonzero_neg_42_pos_cly.txt", df_pos[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")
np.savetxt(img_path+"nn_nonzero_neg_42_neg_cly.txt", df_neg[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")

In [36]:
df_out[df_out['neg']==0]

,i,j,k,neg,predict
720896,0,0,0,0,0.0
720897,0,0,1,0,0.0
720898,0,0,2,0,0.0
720899,0,0,3,0,0.0
720900,0,0,4,0,0.0
...,...,...,...,...,...
901115,43,63,59,0,0.0
901116,43,63,60,0,0.0
901117,43,63,61,0,0.0
901118,43,63,62,0,0.0


In [ ]:
test_pred

In [ ]:
df_out.describe()